In [2]:
import pandas as pd
import numpy as np
from bokeh.io import output_file, show
from bokeh.io.doc import set_curdoc
from bokeh.plotting import figure, curdoc
from bokeh.models import (Grid, Rect, Plot, LinearAxis, ColumnDataSource, HoverTool,
                         LinearColorMapper, ColorBar, LogTicker, BasicTicker, Slider, Button,
                         Div, Dropdown, Select, RadioGroup, CategoricalColorMapper, Legend,
                         FixedTicker, PrintfTickFormatter, CheckboxGroup, Panel, Tabs)
from bokeh.palettes import Spectral3
from bokeh.layouts import row, column
from bokeh.transform import factor_cmap
from anytree import Node
from scipy.stats.kde import gaussian_kde
import math
import colorcet as cc

In [3]:
nfl = pd.read_excel("NFL_Player_Data.xlsx")

In [4]:
nfl.drop(['Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1, inplace=True)
nfl.rename(columns={"Games": "Games Played", "Years": "Contract Years"}, inplace=True)

In [5]:
money_cols = ['Value', 'APY', 'Guaranteed', 'APY as % Of Cap', 
              'Inflated Value', "Inflated APY", "Inflated Guaranteed"]

for i in money_cols:
    nfl[i] = nfl[i].str.replace('[$,%]', '')
    # convert from string to float
    nfl[i] = nfl[i].astype(float)

In [6]:
teams = sorted(nfl['Team'].unique().tolist())
teams.insert(0, "All")

positions = sorted(nfl['Position'].unique().tolist())

compare_vars = ['Age', 'AV', 'Games Played', 'Games Started', 'Pro Bowl', 'AP1', 'Value', 'APY',
               'Guaranteed', 'Contract Years', 'APY as % Of Cap', 'Inflated Value', 'Inflated APY', 
                'Inflated Guaranteed']

players = nfl['Name'].unique().tolist()

In [558]:
def build_combined():
    if team1_select.value != 'All':
        subset1 = nfl[nfl['Team'] == team1_select.value]
        subset1 = subset1[subset1['Position'] == position1_select.value]
    else:
        subset1 = nfl[nfl['Position'] == position1_select.value]
        
    if team2_select.value != 'All':
        subset2 = nfl[nfl['Team'] == team2_select.value]
        subset2 = subset2[subset2['Position'] == position2_select.value]
    else:
        subset2 = nfl[nfl['Position'] == position2_select.value]
        
    
    subset1['key'] = '1'
    subset2['key'] = '2'
    
    combined = subset1.append(subset2, ignore_index=True)
    
    return combined
    

def build_scatter():
    combined = build_combined()
    
    # scale our size variable
    combined['scaled size'] = scale(combined[size_select.value], 1, 50)
    
    # make a mapper for just red and blue
    mapper = CategoricalColorMapper(palette=["red", "blue"], factors=['1', '2'])
    
    title = x_select.value + " vs " + y_select.value
    p = figure(title = title, width=plot_width, height=plot_height,
              tooltips = [('Name', '@Name'), ('Team', '@Team'), ('Year', '@Year')])

    # make legend
    items = []
    names = [position1_select.value, position2_select.value]
    colors = ['red', 'blue']

    for i in range(2):
        items += [(names[i], [p.circle(0,0,fill_color=colors[i], size=.00000001)])]
        
    source = ColumnDataSource(combined)
        
    p.scatter(x_select.value, y_select.value, size='scaled size',
              color={'field': 'key', 'transform': mapper}, fill_alpha=.4,
              source=source,
            )
    
    p.add_layout(Legend(items=items),'right')

    p.xaxis.axis_label = x_select.value
    p.yaxis.axis_label = y_select.value
    position_layout.children[1] = p
    
    print("COMPLETED")
    
def build_density():
    title = 'Density plot'
    p = figure(title = title, width=plot_width, height=plot_height)
    
    combined = build_combined()
    
    s1 = combined[combined['key'] == '1']
    s2 = combined[combined['key'] == '2']
    
    # scale our data from 0 to 100
    s1[x_select.value] = scale(s1[x_select.value], 0, 100)
    s2[x_select.value] = scale(s2[x_select.value], 0, 100)
    
    # create our gaussian probability distribution
    pdf1 = gaussian_kde(s1[x_select.value])
    pdf2 = gaussian_kde(s2[x_select.value])
    
    # create our x and y variables
    x = np.linspace(0, 100, 202)
    y1 = pdf1(x)
    y2 = pdf2(x)
    
    # important code here that makes the ending point the same as starting point.
    # this will create "density" shape.
    x = np.append(x, 100)
    x = np.append(0, x)
    y1 = np.append(y1, 0)
    y1 = np.append(0, y1)
    y2 = np.append(y2, 0)
    y2 = np.append(0, y2)
    
    source = ColumnDataSource(data=dict(x=x, y1=y1, y2=y2))
    
    # use patches widget in bokeh to plot density plot
    p.patch('x', 'y1', color='blue', alpha=0.4, line_color="black", source=source)
    p.patch('x', 'y2', color='orange', alpha=.4, line_color='black', source=source)
    
    p.outline_line_color = None
    p.background_fill_color = "#efefef"

    p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 10)))
#     p.xaxis.formatter = PrintfTickFormatter(format="%d%%")
    p.xaxis.axis_label = 'Scaled(' + x_select.value + ')'

    p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = "#dddddd"
    p.xgrid.ticker = p.xaxis.ticker
    p.yaxis.axis_label = 'Kerndel Density Probability'

    p.axis.minor_tick_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.axis_line_color = None
    
    # make legend
    items = []
    names = [position1_select.value, position2_select.value]
    colors = ['blue', 'orange']

    for i in range(2):
        items += [(names[i], [p.circle(0,0,fill_color=colors[i], size=0.0000001)])]
        
    p.add_layout(Legend(items=items),'right')
    
    position_layout.children[1] = p

    
    print("Completed Density")


def build_data_hist_check(data, xx, bins):
    left = xx[0:bins-1]
    right = xx[1:bins]
    
    df = pd.DataFrame(left, columns=['left'])
    df['right'] = right
    totals = []
    for i in range(len(left)):
        res = len(data[(data[x_select.value] > left[i]) & 
                   (data[x_select.value] < right[i])][x_select.value])
        
        totals.append(res)
        
    df['totals'] = totals
    
    return df

def build_data_hist(data):
    bins = bin_slider.value + 1
    xmin = min(data[x_select.value])
    xmax = max(data[x_select.value])

    xx = np.linspace(xmin, xmax, bins)
    
    left = xx[0:bins-1]
    right = xx[1:bins]
    
    df = pd.DataFrame(left, columns=['left'])
    df['right'] = right
    totals = []
    for i in range(len(left)):
        res = len(data[(data[x_select.value] > left[i]) & 
                   (data[x_select.value] < right[i])][x_select.value])
        
        totals.append(res)
        
    df['totals'] = totals
    
    return df    


def build_histogram():
    title = "Histogram of " + x_select.value
    p = figure(title = title, width=plot_width, height=plot_height)
    
    combined = build_combined()
    
    s1 = combined[combined['key'] == '1']
    s2 = combined[combined['key'] == '2']
    
    if len(checkbox_group.active) == 1:
        bins = bin_slider.value + 1
        xmin = min(s1[x_select.value])
        xmax = max(s1[x_select.value])

        xx = np.linspace(xmin, xmax, bins)
    
        s1df = build_data_hist_check(s1, xx, bins)
        s2df = build_data_hist_check(s2, xx, bins)
        
    else:
        print("OTHER")
        s1df = build_data_hist(s1)
        s2df = build_data_hist(s2)
        
    p.quad(bottom = 0, top=s1df['totals'], right=s1df['right'], left=s1df['left'],
          fill_color='blue', line_color='black', alpha=.4)
    
    p.quad(bottom = 0, top=s2df['totals'], right=s2df['right'], left=s2df['left'],
          fill_color='orange', line_color='black', alpha = .4)
    
    # make legend
    items = []
    names = [position1_select.value, position2_select.value]
    colors = ['blue', 'orange']

    for i in range(2):
        items += [(names[i], [p.circle(0,0,fill_color=colors[i], size=0.0000001)])]
        
    p.add_layout(Legend(items=items),'right')
    
    p.xaxis.axis_label = x_select.value
    p.yaxis.axis_label = "Count"
    
    position_layout.children[1] = p
    
    print("Completed histogram")
    
#     p.quad(bottom = 0, top = s1[x_select.value])
def update_pgraph(event):
    build_tabs()


def build_playerTree(data):
    s1 = data[data['key'] == 1]
    s2 = data[data['key'] == 2]
    
    root = Node(name = "All", value=data['New'].sum(), key='1000')
    root.children = [Node(name = player1_select.value, value=s1['New'].sum(), 
                          key= str(s1['key'].iloc[0])),
                    Node(name = player2_select.value, value=s2['New'].sum(), 
                         key = str(s2['key'].iloc[0]))]
    
    grandchildren1 = []
    grandchildren2 = []
    for i in range(len(s1['Name'])):
        n = Node(name = s1['Name'].iloc[i], value=s1['New'].iloc[i], 
                 year = s1['Year'].iloc[i], team = s1['Team'].iloc[i], key=str(s1['key'].iloc[0]))
        grandchildren1.append(n)
    
    for j in range(len(s2['Name'])):
        n = Node(name = s2['Name'].iloc[j], value=s2['New'].iloc[j],
                 year = s2['Year'].iloc[j], team = s2['Team'].iloc[j], key=str(s2['key'].iloc[0]))
        grandchildren2.append(n)
    
    root.children[0].children = grandchildren1
    root.children[1].children = grandchildren2
    
    return root
        
    
def build_players():
    subset1 = nfl[nfl['Name'] == player1_select.value]
    subset2 = nfl[nfl['Name'] == player2_select.value]
    
    subset1['key'] = 1
    subset2['key'] = 2
    
    combined = subset1.append(subset2, ignore_index=True)
    
    return combined
    
    
def build_tabs():
    p1 = figure(width=plot_width, height=plot_height)
    p2 = figure(width=plot_width, height=plot_height)
    
    s1 = nfl[nfl['Name'] == player1_select.value]
    s2 = nfl[nfl['Name'] == player2_select.value]
    
    if select1down.value != 'None':
        # creating our y axis
        # make sure to replace 0 values so we don't divide by 0
        s1[select1down.value] = s1[select1down.value].replace(0, 1)
        s2[select1down.value] = s2[select1down.value].replace(0,1)
        s1['New'] = s1[select1up.value] / s1[select1down.value]
        s2['New'] = s2[select1up.value] / s2[select1down.value]
        p1.yaxis.axis_label = select1up.value + "/" + select1down.value
        p2.yaxis.axis_label = select1up.value + "/" + select1down.value
        bartitle = "Avg of " + select1up.value + "/" + select1down.value
    
    else:
        s1['New'] = s1[select1up.value]
        s2['New'] = s2[select1up.value]
        p1.yaxis.axis_label = select1up.value
        p2.yaxis.axis_label = select1up.value
        bartitle = "Avg of " + select1up.value
    
    
    # only for radial tree
    s1['key'] = 1
    s2['key'] = 2
    combined = s1.append(s2, ignore_index=True)
    # negative values will mess up radial tree!
    combined['New'] = combined['New'].clip(lower=0)
    ###############################################
    
    # creating index that acts as our x axis
    s1['Index'] = range(1, len(s1['Name']) + 1)
    s2['Index'] = range(1, len(s2['Name']) + 1)
    
    p1.circle(s1['Index'], s1['New'], size = 20, color='blue', alpha=.4)
    p1.circle(s2['Index'], s2['New'], size = 20, color='orange', alpha=.4)
    p1.xaxis.axis_label = "Season"
    
    p2.line(s1['Index'], s1['New'], line_width = 3, color='blue', alpha = .5)
    p2.line(s2['Index'], s2['New'], line_width=3, color='orange', alpha=.5)
    p2.xaxis.axis_label = "Season"
    
    # make legend
    items1 = []
    items2 = []
    names = [player1_select.value, player2_select.value]
    colors = ['blue', 'orange']

    for i in range(2):
        items1 += [(names[i], [p1.circle(0,0,fill_color=colors[i], size=0.0000001)])]
        items2 += [(names[i], [p2.circle(0,0,fill_color=colors[i], size=0.0000001)])]
        
    p1.add_layout(Legend(items=items1),'right')
    p2.add_layout(Legend(items=items2),'right')
    
    # create our average dataframe for bar plot
    avgs = []
    avgs.append(s1['New'].mean())
    avgs.append(s2['New'].mean())
    
    p3 = figure(x_range = [player1_select.value, player2_select.value],
                width=plot_width, height=plot_height,
               title = bartitle)
    
    p3.vbar([player1_select.value, player2_select.value], top = avgs,
           width=.9, fill_alpha = .5, fill_color = ['blue', 'orange'],
           line_alpha = .8, line_color='black')
    
    # BUILD RADIAL TREE
    global root, rows_list
    root = build_playerTree(combined)
    rows_list = []

    p4 = figure(width=plot_width, height=plot_height, x_range=(-3, 3),
                tooltips=[('Name', '@Name'),
                          ('Value', '@Value'),
                          ('Year', '@Year'),
                          ('Team', '@Team')])
    

    initial_dict = {'Name': root.name, 'Value': root.value, 'Team':'NA', 'Year':'NA'}
    root_df = pd.DataFrame([initial_dict])
    root_source = ColumnDataSource(root_df)
    p4.wedge(x=0, y=0, radius=.8, start_angle=0, end_angle=2 * math.pi,
            color="brown", direction="clock", source=root_source, fill_color='brown')

    initial_angle = 0
    build_radial(root, 0, root.height)

    dfr = pd.DataFrame(rows_list)
    radial_source = ColumnDataSource(dfr)

    radial_mapper = CategoricalColorMapper(palette=["orange", "blue"], 
                                           factors=['1', '2'])

    p4.annular_wedge(x=0, y=0, inner_radius='inner', outer_radius='outer',
                    start_angle='start_angle', end_angle='end_angle', source=radial_source,
                   line_color = 'black', fill_color={'field': 'Key', 'transform': radial_mapper},
                   alpha=.7)

    
    tab1=Panel(child=p1, title='circle')
    tab2=Panel(child=p2, title='line')
    tab3=Panel(child=p3, title='bar')
    tab4=Panel(child=p4, title='radial')
    
    table = Tabs(tabs=[tab1, tab2, tab3, tab4])
    
    position_layout.children[1] = table
    
    print("COMPLETED TABS")

In [32]:
def build_radial(node, initial_angle, depth):
    global root, rows_list
    for i in node.children:
        children_angle = 0
        
        dict1 = {}
        inner = .8 * i.depth
        dict1['Name'] = i.name
        dict1['Value'] = i.value
        dict1['Key'] = i.key
        dict1['inner'] = inner
        dict1['outer'] = inner + .8
        dict1['start_angle'] = initial_angle
        dict1['end_angle'] = initial_angle + scale_radial(i.value)
        
        initial_angle += scale_radial(i.value)
        children_angle += scale_radial(i.value)
        
        if len(i.children) > 0:
            dict1['Year'] = 'NA'
            dict1['Team'] = 'NA'
            rows_list.append(dict1)
            build_radial(i, initial_angle - children_angle, depth)
        else:
            # if we are at the end of tree, add the extra information we have
            dict1['Year'] = i.year
            dict1['Team'] = i.team
            rows_list.append(dict1)

            
def build_radial_big(node, initial_angle, depth):
    global rows_list_big
    for i in node.children:
        children_angle = 0
        
        dict1 = {}
        inner = .8 * i.depth
        dict1['Name'] = i.name
        dict1['Value'] = i.value
        dict1['inner'] = inner
        dict1['outer'] = inner + .8
        dict1['start_angle'] = initial_angle
        dict1['end_angle'] = initial_angle + scale_radialtop(i.value)
        dict1['Position'] = i.position
        
        initial_angle += scale_radialtop(i.value)
        children_angle += scale_radialtop(i.value)
        
        rows_list_big.append(dict1)
        
        if len(i.children) > 0:
            build_radial_big(i, initial_angle - children_angle, depth)
            

In [33]:
# updating type of graph
def update_type(attr, old, new):
    if new == 1:  # Density graph
        if old == 0 or old == 2:
            # remove our "size" and "Y Axis" widget option
            position_layout.children[0].children[3].children.pop()
            position_layout.children[0].children[3].children.pop()
            
        position_layout.children[0].children[3].children[0].title = 'Height'
            
        # reset to default plot
        position_layout.children[1] = position_plot
            
    if new == 0:  # Scatterplot
        if old == 2:
            # remove bin slider
            position_layout.children[0].children[3].children.pop()
            position_layout.children[0].children[3].children.pop()
            
        position_layout.children[0].children[3].children.append(y_select)
        position_layout.children[0].children[3].children.append(size_select)
        position_layout.children[0].children[3].children[0].title = 'X Axis'
        
        # reset to default plot
        position_layout.children[1] = position_plot
        
    if new == 2:   # Histogram
        if old == 0:
            # remove our "size" and "Y Axis" widget option
            position_layout.children[0].children[3].children.pop()
            position_layout.children[0].children[3].children.pop()
            position_layout.children[0].children[3].children[0].title = 'Height'
        
        position_layout.children[0].children[3].children.append(bin_slider)
        position_layout.children[0].children[3].children.append(checkbox_group)
            
            
    
def scale(x, minim, maxim):
    X_std = (x - x.min()) / (x.max() - x.min())
    X_scaled = X_std * (maxim - minim) + minim
    return X_scaled


def scale_radial(OldValue):
    OldMax = root.value
    OldMin = 0
    OldRange = (OldMax - OldMin)
    NewRange = 2 * math.pi   # make sure we are in radians
    NewValue = (((OldValue - OldMin) * NewRange) / OldRange)
    
    return NewValue

def scale_radialtop(OldValue):
    OldMax = root_big.value
    OldMin = 0
    OldRange = (OldMax - OldMin)
    NewRange = 2 * math.pi   # make sure we are in radians
    NewValue = (((OldValue - OldMin) * NewRange) / OldRange)
    
    return NewValue
    

def update_graph(event):
    if radio_group.active == 0:
        build_scatter()
        
    elif radio_group.active == 1:
        build_density()
        
    elif radio_group.active == 2:
        build_histogram()
     
    
def to_menu():
    global position_layout
    # clear everything in layout
    for i in range(len(position_layout.children)):
        position_layout.children.pop()
        
    plot = figure(title=None, width=plot_width, height=plot_height,
        min_border=0, y_range=(0,10))

    position_button = Button(label = "Compare Positions")
    position_button.on_click(comparePositions)
    player_button = Button(label = 'Compare Players')
    player_button.on_click(comparePlayers)
    cool_button = Button(label = 'Compare All Positions')
    cool_button.on_click(coolGraphs)

    div = Div(text="""<b>Final Project: Comparing Performance and Compensation in NFL Player Data </b> 
                by Davin Miller""",
             width=250, height=50)

    div2 = Div(text="""Please Select an option from below:""",
              width=225, height=25)
    
    menu_layout = [column(div, div2, position_button, player_button, cool_button)]
    position_layout.children = menu_layout
    
#     position_layout = menu_layout
    
    print("TO MENU")


def comparePositions():
    # change radio group back to default option
    radio_group.active = 0
    
    layout1 = row(team1_select, team2_select)
    layout2 = row(position1_select, position2_select)
    layout3 = row(x_select, y_select, size_select)
    left_layout = column(radio_group, layout2, layout1, layout3, update_button, menu_button)
    
    # set new layout and change children
    new_layout = [left_layout, position_plot]
    position_layout.children = new_layout
    
def update_players1(attr, old, new):
    global player1_select, team_player_select1
    player1_select.options = nfl[nfl['Team'] == team_player_select1.value]['Name'].unique().tolist()

def update_players2(attr, old, new):
    global player2_select, team_player_select2
    player2_select.options = nfl[nfl['Team'] == team_player_select2.value]['Name'].unique().tolist()
    

def comparePlayers(): 
    position_plot = figure(title = None, width=plot_width, height=plot_height,
                      min_border=0, y_range=(0,10))
    
    layout1 = row(team_player_select1, team_player_select2)
    layout2 = row(player1_select, player2_select)
    select_layout = row(select1up, select1down)
    
    layout3 = column(layout1, layout2, select_layout, update_button_players, menu_button)
    
    position_layout.children = [layout3, position_plot]


def build_coolGraph(event):
    if graph_select.value == 'Densities':
        position_layout.children[1] = build_densities(var_select.value)
    
    else:
        build_topradial(var_select.value)
        
    
def coolGraphs():
    position_plot = figure(title = None, width=plot_width, height=plot_height,
                      min_border=0, y_range=(0,10))
    
    
    compareAll_button = Button(label='Update Graph', button_type='success')
    compareAll_button.on_click(build_coolGraph)
    
#     position_button_av = Button(label = 'All Positions Density Comparison for AV')
#     position_button_av.on_click(positionAV)
#     position_button_apy = Button(label = 'All Positions Density Comparison for Inflated APY')
#     position_button_apy.on_click(positionAPY)
    
#     radial_top = Button(label = 'Top 100 players for AV')
#     radial_top.on_click(topAV)
#     radial_top2 = Button(label = 'Top 100 players for Inflated APY')
#     radial_top2.on_click(topAPY)
    
    layout = row(graph_select, var_select)
    layout2 = column(layout, compareAll_button, menu_button)
    
    position_layout.children = [layout2, position_plot]
    

In [58]:
def ridge(category, data, scale=20):
    return list(zip([category]*len(data), scale*data))

def build_densities(var):
    palette = [cc.rainbow[i*14] for i in range(len(positions))]

    x = np.linspace(0, 100, 202)
    x = np.append(x, 100)
    x = np.append(0, x)

    source = ColumnDataSource(data=dict(x=x))

    p = figure(y_range=positions, width=plot_width, height=plot_height)
    
    pos_iter = positions.copy()
    if var == 'AP1' or var == 'Pro Bowl':
        pos_iter.remove('long-snapper')
        pos_iter.remove('fullback')

    for i, pos in enumerate(reversed(pos_iter)):
        
        yy = nfl[nfl['Position'] == pos].replace([np.inf, -np.inf], np.nan)
        yy.dropna(inplace=True)
        yy = yy[var]
        yy = scale(yy, 0, 100)
        try:
            pdf = gaussian_kde(yy) 
        except Exception:
            print(pos)
            print(yy)
        y = ridge(pos, pdf(x))
        y.append((pos, 0))
        y.insert(0, (pos, 0))
        source.add(y, pos)
        p.patch('x', pos, color=palette[i], alpha=0.6, line_color="black", source=source)

    p.outline_line_color = None
    p.background_fill_color = "#efefef"

    p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 10)))
    p.xaxis.formatter = PrintfTickFormatter(format="%d%%")
    p.xaxis.axis_label = 'Scaled(' + var + ')'

    p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = "#dddddd"
    p.xgrid.ticker = p.xaxis.ticker

    p.axis.minor_tick_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.axis_line_color = None

    p.y_range.range_padding = 0.12
    
    return p


def build_toptree(data, val):
    root = Node(name='All', value = data[val].sum())
    position_group = data.groupby(by='Position', as_index=False).sum()
    
    children = []
    # not all positions will be in top 100 list
    unique_pos = position_group['Position'].unique().tolist()
    for i in unique_pos:
        carVal = position_group[position_group['Position'] == i][val].iloc[0]
        childNode = Node(name=i, value=carVal, position=i)
        children.append(childNode)
        
    root.children = children
    
    for child in root.children:
        pos = child.name
        names = data[data['Position'] == pos]
        names_group = names.groupby('Name', as_index=False).sum()
        grandchildren = []
        
        templist = list(zip(names_group['Name'], names_group[val]))
        for i in templist:
            grandchildNode = Node(name = i[0], value=i[1], position=pos)
            grandchildren.append(grandchildNode)
            
        child.children = grandchildren
        
    return root
    

def build_topradial(val):
    global rows_list_big, root_big
    p = figure(width=plot_width, height=plot_height, x_range=(-3, 3),
                tooltips=[('Name', '@Name'),
                          ('Value', '@Value')])
    
    
    nfl_group = nfl.groupby('Name', as_index=False).sum()
    top100Names = nfl_group.sort_values(val, ascending=False)['Name'].unique()[0:100].tolist()
    top100df = nfl[nfl['Name'].isin(top100Names)]
    root_big = build_toptree(top100df, val)
    
    initial_dict = {'Name': root_big.name, 'Value': root_big.value}
    root_df = pd.DataFrame([initial_dict])
    root_source = ColumnDataSource(root_df)
    
    p.wedge(x=0, y=0, radius=.8, start_angle=0, end_angle=2 * math.pi,
            color="brown", direction="clock", source=root_source, fill_color='brown')
    
    initial_angle = 0
    rows_list_big = []
    
    build_radial_big(root_big, 0, root_big.height)
    
    dfr = pd.DataFrame(rows_list_big)
    radial_source = ColumnDataSource(dfr)
    
    # create color palette
    pos_palette = [cc.rainbow[i*14] for i in range(len(positions))]
    pos_mapper = CategoricalColorMapper(palette=pos_palette, factors=positions)
    
    p.annular_wedge(x=0, y=0, inner_radius='inner', outer_radius='outer',
                    start_angle='start_angle', end_angle='end_angle', source=radial_source,
                   line_color = 'black', fill_color={'field': 'Position', 'transform': pos_mapper},
                   alpha=.7)
    
#     show(p)
    position_layout.children[1] = p
    
    

def positionAV(event):
    position_layout.children[1] = build_densities('AV')
    
def positionAPY(event):
    position_layout.children[1] = build_densities('Inflated APY')
    
def topAV(event):
    build_topradial('AV')

def topAPY(event):
    build_topradial('Inflated APY')

In [28]:
# Plot logistics
plot_width = 960
plot_height = 800

In [29]:
# PLAYER WIDGETS
teams_players = teams[1:len(teams)]
team_player_select1 = Select(title='Select Team 1', value="NE", options=teams_players)
team_player_select2 = Select(title='Select Team 2', value='IND', options=teams_players)

player1_select = Select(title='Select Player 1', value = "Tom Brady", options = players)
player2_select = Select(title='Select Player 2', value = "Peyton Manning", options = players)

update_players1('1','1','1')
update_players2('1','1','1')

team_player_select1.on_change('value', update_players1)
team_player_select2.on_change('value', update_players2)

# provide *none* option in denominator
compare_vars_none = compare_vars.copy()
compare_vars_none.insert(0, "None")

# select compare vars
select1up = Select(title = 'numerator', value="AV", options=compare_vars)
select1down = Select(title = 'denominator', value="None", options=compare_vars_none)

# select2up = Select(title= 'numerator 2', options=compare_vars_none)
# select2down = Select(title = 'denominator 2', options=compare_vars_none)

NameError: name 'update_players1' is not defined

In [30]:
# POSITION WIDGETS
position_plot = figure(title = None, width=plot_width, height=plot_height,
                      min_border=0, y_range=(0,10))


# SELECT BUTTONS FOR TEAM, POSITIONS, AXIS
position1_select = Select(title='Select Position 1', value='quarterback', options = positions)
position2_select = Select(title='Select Position 2', value='center', options=positions)

# team1_select.on_change('value', update_team1)
team1_select = Select(title='Select Team 1', value="All", options=teams)
team2_select = Select(title='Select Team 2', value="All", options=teams)

x_select = Select(title='X Axis', value='AV', options=compare_vars, width=197)
y_select = Select(title='Y Axis', value='Pro Bowl', options=compare_vars, width=197)
size_select = Select(title='Size', value='Games Started', options=compare_vars, width=197)
bin_slider = Slider(start=1, end=25, value=5,
                   step=1, title='Num Bins')

# SELECT FOR COOL GRAPHS
graph_select = Select(title='Select Type of Graph', value='Densities', 
                      options=['Densities', 'Radial (Top 100)'])
var_select = Select(title='Select variable', value="AV", options=compare_vars)

# CHECKBOX (RADIO) WIDGETS
radio_group = RadioGroup(labels=["Scatter", "Density", "Histogram"], active=0)
radio_group.on_change('active', update_type)

# Checkbox for Histogram
checkbox_group = CheckboxGroup(labels=['Aligned Bins'], active=[0],width=80)

# BUTTONS
update_button = Button(label="UPDATE GRAPH", button_type="success")
update_button.on_click(update_graph)
update_button_players = Button(label = 'UPDATE GRAPH', button_type='success')
update_button_players.on_click(update_pgraph)

menu_button = Button(label = 'MAIN MENU', button_type = 'danger')
menu_button.on_click(to_menu)

layout1 = row(team1_select, team2_select)
layout2 = row(position1_select, position2_select)
layout3 = row(x_select, y_select, size_select)
left_layout = column(radio_group, layout2, layout1, layout3, update_button, menu_button)
position_layout = row(left_layout, position_plot)
show(position_layout)
curdoc().add_root(position_layout)

# we want to start with main menu
to_menu()

NameError: name 'update_type' is not defined

In [7]:
output_file("networkx_graph.html")